# Linear regression non-numeric feature combined with numeric features

This notebook contains code to train and evaluate a linear regression model using a single non-numeric feature, two numeric features and numeric label.
There is a good post on [Stack overflow](https://stackoverflow.com/questions/34007308/linear-regression-analysis-with-string-categorical-features-variables) on the topic of string and categorical values as features

One-hot encoding will be used to convert the string feature into numerical features

Import the required python libraries
- **pandas** contains the dataframe object and a number of useful methods for manipulating and querying data contained in a dataframe
- **numpy** contains a number of useful mathematical operations including some that are helpful when evaluating accuracy of trained models
- **Scikitlearn train_test_split** splits data into training and test sets
- **scikitlearn LinearRegression** used to train a linear regression model
- **scikitlearn metrics** used to calculate metrics such as Mean Squared Error, helpful when evaluating accuracy of trained models
- **matplotlib pyplot** used to plot graphs

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
#import statsmodels.api as sm
import matplotlib.pyplot as plt

Read flight data from csv file into Pandas dataframe

In [3]:
flight_df=pd.read_csv('all_flights.csv') # Read csv file into flight_df dataframe
flight_df.shape                          # Display shape of array to see how many rows and columns are in the dataframe

(616101, 17)

Display the first 10 rows in the dataset to make sure data looks like it imported correctly

In [4]:
flight_df.head()                         # Displays the top 10 rows from flight_df dataframe

,FL_DATE,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,Unnamed: 16
0,2018-10-01,WN,N221WN,802,ABQ,BWI,905,903.0,-2.0,1450,1433.0,-17.0,225.0,210.0,197.0,1670.0,NaN
1,2018-10-01,WN,N8329B,3744,ABQ,BWI,1500,1458.0,-2.0,2045,2020.0,-25.0,225.0,202.0,191.0,1670.0,NaN
2,2018-10-01,WN,N920WN,1019,ABQ,DAL,1800,1802.0,2.0,2045,2032.0,-13.0,105.0,90.0,80.0,580.0,NaN
3,2018-10-01,WN,N480WN,1499,ABQ,DAL,950,947.0,-3.0,1235,1223.0,-12.0,105.0,96.0,81.0,580.0,NaN
4,2018-10-01,WN,N227WN,3635,ABQ,DAL,1150,1151.0,1.0,1430,1423.0,-7.0,100.0,92.0,80.0,580.0,NaN


Display the names of the columns in the dataframe and their datatypes

In [5]:
flight_df.dtypes  # Displays the names and data types of the columns in the dataframe

FL_DATE                 object
OP_UNIQUE_CARRIER       object
TAIL_NUM                object
OP_CARRIER_FL_NUM        int64
ORIGIN                  object
DEST                    object
CRS_DEP_TIME             int64
DEP_TIME               float64
DEP_DELAY              float64
CRS_ARR_TIME             int64
ARR_TIME               float64
ARR_DELAY              float64
CRS_ELAPSED_TIME       float64
ACTUAL_ELAPSED_TIME    float64
AIR_TIME               float64
DISTANCE               float64
Unnamed: 16            float64
dtype: object

Create a new dataframe containing ONLY the columns we want to use as features and labels.
This saves us spending time cleaning up data we are not using to train our model.
This model will use OP_UNIQUE_CARRIER (the airline operating the flight) as a feature to predict the value of the label ARR_DELAY

In [6]:
# Create a new dataframe containing only OP_UNIQUE_CARRIER, DEP_DELAY, DISTANCE and ARR_DELAY
min_flight_data_df = flight_df[['OP_UNIQUE_CARRIER','DEP_DELAY','DISTANCE','ARR_DELAY']] 

min_flight_data_df.shape                                          # Display the shape of the dataframe as a quick check to 
                                                                  # ensure dataframe has expected number of columns and rows

(616101, 4)

Get rid of rows containing NaN/missing values

In [7]:
no_missing_values_df = min_flight_data_df.dropna(axis=0,how='any')  # Use dropna to remove rows with NaN in any column
no_missing_values_df.shape                                          # Display shape to see how many rows were removed

(610334, 4)

There does seem to be a relation between carrier and delay times. So maybe I can train a model using OP_UNIQUE_CARRIER as a feature.

OP_UNIQUE_CARRIER is not numeric so I need to make it numeric, basically I want to know iif a flight is operated by a particular airline is it more likely to be late. 

I need to make a dummy column for each possible airline i.e. AIRCANADA 1/0 WESTJET 1/0 AIRTRANSAT 1/0 for every row. a value of 1 means the flight was operated by that carrier. Then I can include those columns as features when I train my model.

The *[get_dummies](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html)* method in pandas will return a dataframe of dummy columns for a given column.

In [9]:
dummy_df = pd.get_dummies(no_missing_values_df['OP_UNIQUE_CARRIER'])
dummy_df.head

<bound method NDFrame.head of         9E  AA  AS  B6  DL  EV  F9  G4  HA  MQ  NK  OH  OO  UA  WN  YV  YX
0        0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   0   0
1        0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   0   0
2        0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   0   0
3        0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   0   0
4        0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   0   0
...     ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
616096   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   0
616097   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   0
616098   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   0
616099   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   0
616100   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   0

[610334 rows x 17 columns]>

Create one dataframe containing only the features. 
The OP_UNIQUE_CARRIER features are in the dummy dataframe
The DEP_DELAY and DISTANCE features are in our no_missing_values_df

We need to extract the DEP_DELAY and DISTANCE features into their own dataframe then combine the two dataframes into
one single dataframe using teh pandas **concat** method

Then we need to create one dataframe containing only the label (ARR_DELAY) 
If either of these has only one column you must reshape the dataframe to -1,1

In [10]:
# Create a dataFrame containing the feature columns
# combine the dummy dataframe with a dataframe containing the numeric features
numeric_features = no_missing_values_df[['DISTANCE','DEP_DELAY']]
all_features_df = pd.concat([dummy_df, numeric_features], axis=1)
X = all_features_df

In [11]:
# Create a DataFrame containing the labels
# Reshape to -1,1 if only containing a single column
y = no_missing_values_df['ARR_DELAY'].values.reshape(-1,1)

Split the data into two datasets, one for training the model, one for testing the model

scikitlearn method [train_test_split]( 
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) 
- **X** is the dataframe containing our features
- **Y** is the dataframe containing our label
- **test_size** determines what fraction of the data is put into the test dataframe
- **random_state** defaults to a random number, by specifying a specific number I ensure the split I generate is reproducible. That way if I make changes I know changes in accuracy are not caused by different rows used as training or test data.

In [12]:
#Split into training and test data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30, random_state=42)

Check the size of your test and training datasets (i.e. how many rows in each)

In [13]:
X_train.shape

(427233, 19)

In [14]:
X_test.shape

(183101, 19)

Train the model using scikitlearn [LinearRegression]( 
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

Your feature must be numeric that's why we created the dummy columns for OP_UNIQUE_CARRIER

Your training data cannot contain any missing values in rows.

In [15]:
regressor = LinearRegression()     # Create a scikit learn LinearRegression object
regressor.fit(X_train, y_train)    # Use the fit methong to train the model using your training data

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Let's see what arrival delays are predicted for our test data by using the **predict** method of our linearRegression object

In [16]:
y_pred = regressor.predict(X_test)

Let's compare the actual arrival delay times and the predicted arrival delay times

In [17]:
#combine the two 1D numpy arrays into a 2D array
combined = np.hstack((y_test,y_pred))
#Convert to a DataFrame
accuracy_df = pd.DataFrame(combined,columns=['Actual','Predicted'])
accuracy_df.head

<bound method NDFrame.head of         Actual  Predicted
0        -13.0 -12.181343
1        -24.0  -4.946919
2        100.0  65.085465
3         -8.0  -8.693103
4        -19.0 -13.748650
...        ...        ...
183096    48.0  57.503282
183097   -24.0  -7.836193
183098    20.0  23.317080
183099    47.0  47.702381
183100   -11.0  -5.158120

[183101 rows x 2 columns]>

We can do some calculations to get a sense of overall accuracy

In [18]:
print('Mean absolute error: ',metrics.mean_absolute_error(y_test,y_pred))
# For comparison, when we trained with only DEP_DELAY as a feature we get 
# Mean absolute error:  9.033505526602951
# For comparison, when we trained with DEP_DELAY and DISTANCE as a feature we get
# Mean absolute error:  9.01393034185557

Mean absolute error:  8.909834662395848


print('Mean Squared Error:', metrics.mean_squared_error(y_test,y_pred))

In [19]:
print('Mean Squared Error:', metrics.mean_squared_error(y_test,y_pred))
# For comparison, when we trained with only DEP_DELAY as a feature we get 
# Mean Squared Error: 163.26825261343365
# For comparison, when we trained with DEP_DELAY and DISTANCE as a feature we get
# Mean Squared Error: 162.04368013600566

Mean Squared Error: 158.60062293292938


In [20]:
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test,y_pred)))
# For comparison, when we trained with only DEP_DELAY as a feature we get 
# Mean Squared Error: 163.26825261343365
# For comparison, when we trained with DEP_DELAY and DISTANCE as a feature we get
# Root Mean Squared Error: 12.729637863506003

Root Mean Squared Error: 12.593673925147076


Check coefficients for our attributes

In [55]:
print(regressor.coef_)
print(X_train.columns)

# Loop through to display these in a more readable way
for index in range(len(X_train.columns)):
    print (str(X_train.columns[index]) + ': ' + str(regressor.coef_[0,index])  )
    

[[-1.40729462e+00  6.00823198e-01  2.23344031e+00  6.87136382e-01
  -2.97148497e+00  1.82041345e+00 -4.73946776e-01 -6.07887942e-01
   2.68212490e+00  4.77563583e-01 -2.01527514e+00 -2.71641224e+00
  -3.44648343e-02  1.48651590e+00 -2.69327792e+00  2.58568164e+00
   3.46345075e-01 -2.19044789e-03  1.00478987e+00]]
Index(['9E', 'AA', 'AS', 'B6', 'DL', 'EV', 'F9', 'G4', 'HA', 'MQ', 'NK', 'OH',
       'OO', 'UA', 'WN', 'YV', 'YX', 'DISTANCE', 'DEP_DELAY'],
      dtype='object')
9E: -1.40729462239299
AA: 0.6008231984182523
AS: 2.2334403146835338
B6: 0.6871363823660214
DL: -2.9714849674095416
EV: 1.8204134458169159
F9: -0.4739467762156722
G4: -0.6078879416830151
HA: 2.6821248991647404
MQ: 0.4775635827815127
NK: -2.015275136285377
OH: -2.716412239581445
OO: -0.034464834340939325
UA: 1.486515901010831
WN: -2.693277919884317
YV: 2.585681638316784
YX: 0.3463450752357463
DISTANCE: -0.0021904478861746695
DEP_DELAY: 1.0047898718148953


When we combine multiple features we start to see changes in the coefficients across airlines. 
- DEP_DELAY still shows a high coefficient and is a large determinign factor for ARR_DELAY
- The values for OP_UNIQUE_CARRIER are larger than the values for DISTANCE, so it seems the carrier is a bigger influence in predciting arrival delay than distance

- Delta flights (DL) seem less likely to be late since they have a larger negative value
- Mesa airlines (YV) seems more likely to be late since they have a larger positive value 

